# ETL Pipeline for Pre-Processing the Files using Cassandra

## Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

## Extract and Load: Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):   
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob('/home/workspace/event_data/*.csv')

/home/workspace


## Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        # extracting each data row one by one and append it        
        for line in csvreader:
            # append the filename to full_data_rows_list 
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Data validation: checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Transform : Starting with Apache Cassandra analysis

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE CassandraProject 
                    WITH REPLICATION = {
                        'class' : 'SimpleStrategy', 'replication_factor' : 1 
                    }""")
except Exception as e:
    print(e)


Keyspace 'cassandraproject' already exists


#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandraproject')
except Exception as e:
    print(e)

#### Now, creating tables that needs to be run on queries. With Apache Cassandra, we model the database tables on the queries we want to run.

#### Queries: 

#### 1. Select artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### For the above query, we will model our table with sessionId and itemInSession columns as our Primary Key since it gives us a good partitioning into our database to get the results of the query in an optimised way.

In [8]:
# Query 1:
# Create table "artist_details" with columns sessionID, itemInSession, userId, artist, song_title, song_length
query = "create table if not exists artist_details"
query = query + "(sessionID varchar, itemInSession varchar, userId varchar, \
                    artist varchar, song_title varchar, song_length varchar, \
                    PRIMARY KEY (sessionID, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Insert data into "artist_details" table from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    # skip header
    next(csvreader) 
    for line in csvreader:
        # Insert data into artist_details table for Query 1.
        query = "INSERT INTO artist_details (sessionID, itemInSession, userID, \
                 artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[8], line[3], line[10], line[0], line[9], line[5]))

##### Select and verify that the data have been inserted into the table

In [10]:
rows = session.execute("Select artist, song_title, song_length from artist_details where sessionID = '338' and itemInSession = '4'")

for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


#### 2. Select name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### For the above query, we will model our table with userID and sessionID as the Primary Key. And itemInSession, artist and song_title as the clustering cloumns. As we need our data sorted by itemInSession columns, we have listed it as the first column in clustering columns. 

In [11]:
# Create table artist_details_with_user_info with columns userID, sessionID, itemInSession, artist,
# song_title, firstName, lastName, 
query = "create table if not exists artist_details_with_user_info"
query = query + "(userID varchar, sessionID varchar,itemInSession varchar, artist varchar,\
        song_title varchar, firstName varchar, lastName varchar, \
        PRIMARY KEY ((userID, sessionID), itemInSession, artist, song_title))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:
# Insert data into artist_details_with_user_info table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details_with_user_info \
        (userID, sessionID, itemInSession, artist, song_title, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))

##### Select and verify that the data have been inserted into the table

In [13]:
rows = session.execute("Select userID, sessionID, itemInSession, artist, \
                       song_title, firstName, lastName from artist_details_with_user_info \
                       where userID = '10' and sessionID = '182'")
# Print the needed columns here
for row in rows:
    print (row.artist, row.song_title,row.firstname, row.lastname, row.iteminsession)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


#### 3. Select every user's name (first and last) in music app history who listened to the song 'All Hands Against His Own'

##### For the above query, we will model our table with userID and song_title columns as our Primary Key since it gives us a good partitioning into our database to get the results of the query in an optimised way.

In [20]:
# Create a table usersListeningToASong with columns userID, song_title, fisrtName, lastName.

query = "create table if not exists usersListeningToASong"
query += "(song_title text, userID varchar, firstName varchar, lastName varchar, \
            PRIMARY KEY(song_title, userID))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [21]:
# Insert data into usersListeningToASong table.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usersListeningToASong (song_title, userID, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[10], line[1], line[4]))

##### Select and verify that the data have been inserted into the table

In [23]:
rows = session.execute("select song_title, firstname, lastname from usersListeningToASong\
                        where song_title = 'All Hands Against His Own'")

for row in rows:
    print (row.song_title, row.firstname, row.lastname)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions

session.execute("drop table artist_details")
session.execute("drop table artist_details1")
session.execute("drop table usersListeningToASong")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()